<a href="https://colab.research.google.com/github/armandoscarpelin/Reconhecimento-de-Face/blob/main/Detector_de_Faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Passo 1: Instalar as bibliotecas necessárias
!pip install opencv-python opencv-python-headless matplotlib

# Passo 2: Importar as bibliotecas necessárias
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

# Passo 3: Solicitar upload da imagem de referência (Diego)
print("Faça o upload da imagem de referência do Diego:")
uploaded_diego = files.upload()

# Passo 4: Solicitar upload da imagem de teste (para localizar rostos)
print("Agora, faça o upload da imagem de teste (aquela onde você quer localizar rostos):")
uploaded_test = files.upload()

# Passo 5: Verificar se ambos os arquivos foram carregados
diego_image_path = list(uploaded_diego.keys())[0]  # Imagem de referência
test_image_path = list(uploaded_test.keys())[0]  # Imagem de teste

# Carregar as imagens
diego_image = cv2.imread(diego_image_path)
test_image = cv2.imread(test_image_path)

# Passo 6: Convertendo as imagens para escala de cinza
diego_gray = cv2.cvtColor(diego_image, cv2.COLOR_BGR2GRAY)
test_gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

# Passo 7: Carregar o classificador em cascata para detecção de rostos (OpenCV)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Passo 8: Detectar rostos nas imagens
diego_faces = face_cascade.detectMultiScale(diego_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
test_faces = face_cascade.detectMultiScale(test_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Passo 9: Selecione o rosto de Diego (assumindo que seja o único rosto na imagem de referência)
if len(diego_faces) == 0:
    raise ValueError("Nenhum rosto detectado na imagem de Diego.")
diego_face = diego_faces[0]  # Assumimos que há apenas um rosto (o de Diego)
x, y, w, h = diego_face
diego_face_roi = diego_gray[y:y+h, x:x+w]

# Passo 10: Comparar os rostos na imagem de teste
results = []
for (x, y, w, h) in test_faces:
    test_face_roi = test_gray[y:y+h, x:x+w]

    # Usar método simples de comparação de rostos (por exemplo, histograma)
    diego_hist = cv2.calcHist([diego_face_roi], [0], None, [256], [0, 256])
    test_hist = cv2.calcHist([test_face_roi], [0], None, [256], [0, 256])

    # Normalizar os histogramas
    cv2.normalize(diego_hist, diego_hist)
    cv2.normalize(test_hist, test_hist)

    # Comparar os histogramas (menor valor de comparação significa maior similaridade)
    comparison = cv2.compareHist(diego_hist, test_hist, cv2.HISTCMP_CORREL)

    # Definir um limiar para considerar um rosto como "Diego"
    if comparison > 0.8:  # Ajuste o limiar conforme necessário
        results.append("Diego")
    else:
        results.append("Outro Resultado")

# Passo 11: Exibir os resultados e desenhar os rostos na imagem de teste
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))

# Passo 12: Desenhar as caixas delimitadoras e adicionar os nomes
for i, (x, y, w, h) in enumerate(test_faces):
    if results[i] == "Diego":
        # Caixa verde para o rosto de Diego
        ax.add_patch(plt.Rectangle((x, y), w, h, fill=False, edgecolor='green', linewidth=2))
        ax.text(x, y - 10, results[i], color='green', fontsize=12)
    else:
        # Caixa vermelha para outros rostos
        ax.add_patch(plt.Rectangle((x, y), w, h, fill=False, edgecolor='red', linewidth=2))
        ax.text(x, y - 10, results[i], color='red', fontsize=12)

plt.axis('off')
plt.show()
